In [1]:
%config Completer.use_jedi = False

import configparser
config = configparser.ConfigParser()
config.read('agg-config.txt')


time_step = int(config.get('Configuration', 'time_step'))
time_step2 = int(config.get('Configuration', 'time_step2'))

window_size = int(config.get('Configuration', 'window_size'))
processed_data_folder = config.get('Configuration', 'processed_data_folder')
raw_data_folder = config.get('Configuration', 'raw_data_folder')

ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')


activities=config.get('Configuration', 'learn_activities')
activities = [(item.strip()) for item in activities.split(',')]

cross_days=config.get('Configuration', 'cross_days')
cross_days = [(item.strip()) for item in cross_days.split(',')]

sensors=config.get('Configuration', 'sensors')
sensors = [(item.strip()) for item in sensors.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))

users_day={}
for cross_day in cross_days:
    users_day[cross_day]=config.get('Configuration', cross_day)
    users_day[cross_day] = [(item.strip()) for item in users_day[cross_day].split(',')]
print(users_day)

{'2024-07-05': ['0001', '3d57', 'ed9c'], '2024-07-06': ['0001', '3d57', 'ed9c'], '2024-07-07': ['0001', '3d57', 'ed9c'], '2024-07-18': ['16fe', 'f02e', 'ed9c'], '2024-07-19': ['16fe', 'f02e', 'ed9c'], '2024-07-20': ['16fe', 'f02e', 'ed9c'], '2024-07-24': ['16fe', '5b66', 'ed9c'], '2024-07-25': ['16fe', '5b66', 'ed9c'], '2024-07-26': ['16fe', '5b66', 'ed9c'], '2024-07-27': ['16fe', '5b66', 'ed9c'], '2024-07-28': ['16fe', '5b66', 'ed9c'], '2024-07-29': ['16fe', '5b66', 'ed9c'], '2024-07-30': ['16fe', '5b66', 'ed9c'], '2024-07-31': ['16fe', '5b66', 'ed9c']}


In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(t0,tN,ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(t0,tN,tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


def day_time0(t0,tN,ti):
    return day_time(ti)-day_time(t0)+1

def relT(t0,tN,ti):
    return (int)((ti-t0)/time_step2)

def sizeT(t0,tN):
    return relT(t0,tN,tN)+1


def initTime(ini_date,end_date):
    tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())

    t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
    print (ini_date,"Init date is", t0, "day:",day_time(t0,tN,t0), time2str(t0,tN,t0))
    print (end_date,"End date is", tN, "day:",day_time(t0,tN,tN), time2str(t0,tN,tN))

    day=list(range(day_time(t0,tN,t0),day_time(t0,tN,tN)+1))[0]
    return day,t0,tN
    
initTime(ini_date,end_date)


def getStrDatefrom(day_number):
    # Reference date (1970-01-01)
    reference_date = datetime(1970, 1, 1)
    
    # Compute the date corresponding to the day number
    resulting_date = reference_date + timedelta(days=day_number)
    
    return resulting_date.strftime("%Y-%m-%d")


2024-07-31 02:00:00 Init date is 1722384000 day: 19935 2024-07-31 02:00:00
2024-08-01 01:59:59 End date is 1722470399 day: 19936 2024-08-01 01:59:59


In [3]:
from datetime import datetime, timedelta

days=[]
t0N_days={}

for cross_day in cross_days:
    print(cross_day)
    # Convertir las fechas a objetos datetime
    ini_date_dt = datetime.strptime(ini_date, "%Y-%m-%d %H:%M:%S")
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S")
    cross_day_dt = datetime.strptime(cross_day, "%Y-%m-%d")

    # Reemplazar el día manteniendo la hora original
    new_ini_date = cross_day_dt.replace(hour=ini_date_dt.hour, minute=ini_date_dt.minute, second=ini_date_dt.second)
    new_end_date = cross_day_dt.replace(hour=end_date_dt.hour, minute=end_date_dt.minute, second=end_date_dt.second) + timedelta(days=1)

    # Convertir las fechas de nuevo a cadena si es necesario
    new_ini_date_str = new_ini_date.strftime("%Y-%m-%d %H:%M:%S")
    new_end_date_str = new_end_date.strftime("%Y-%m-%d %H:%M:%S")

    print(f"New ini_date: {new_ini_date_str}")
    print(f"New end_date: {new_end_date_str}")
    day,t0,tN=initTime(new_ini_date_str,new_end_date_str)
    days.append(day)
    t0N_days[day]=(t0,tN)
    

print(days)
print(t0N_days)



2024-07-05
New ini_date: 2024-07-05 02:00:00
New end_date: 2024-07-06 01:59:59
2024-07-05 02:00:00 Init date is 1720137600 day: 19909 2024-07-05 02:00:00
2024-07-06 01:59:59 End date is 1720223999 day: 19910 2024-07-06 01:59:59
2024-07-06
New ini_date: 2024-07-06 02:00:00
New end_date: 2024-07-07 01:59:59
2024-07-06 02:00:00 Init date is 1720224000 day: 19910 2024-07-06 02:00:00
2024-07-07 01:59:59 End date is 1720310399 day: 19911 2024-07-07 01:59:59
2024-07-07
New ini_date: 2024-07-07 02:00:00
New end_date: 2024-07-08 01:59:59
2024-07-07 02:00:00 Init date is 1720310400 day: 19911 2024-07-07 02:00:00
2024-07-08 01:59:59 End date is 1720396799 day: 19912 2024-07-08 01:59:59
2024-07-18
New ini_date: 2024-07-18 02:00:00
New end_date: 2024-07-19 01:59:59
2024-07-18 02:00:00 Init date is 1721260800 day: 19922 2024-07-18 02:00:00
2024-07-19 01:59:59 End date is 1721347199 day: 19923 2024-07-19 01:59:59
2024-07-19
New ini_date: 2024-07-19 02:00:00
New end_date: 2024-07-20 01:59:59
2024-07-1

In [4]:
import os
import pandas as pd
len_users=3
loc_user=np.zeros((len(activities),len_users,len(days),sizeT(t0,tN)))
print(loc_user.shape)
for ax,activity in enumerate(activities):
    print("\t",activity)
    for dx,day in enumerate(days):
        day_str=getStrDatefrom(day)
        users=users_day[day_str]
        print(day,users)
        for ux,user in enumerate(users):
            print(processed_data_folder+"/DAY_"+str(day)+"/act/loc."+user+"."+activity+".tsv")
            series=read_csv(processed_data_folder+"/DAY_"+str(day)+"/act/loc."+user+"."+activity+".tsv", sep="\t",parse_dates=True, header=None, names=["time","loc"])
            series=series["loc"].values
            print(day,series.shape)
            loc_user[ax][ux][dx]=series
    


(8, 3, 14, 1440)
	 toileting
19909 ['0001', '3d57', 'ed9c']
./data-har//DAY_19909/act/loc.0001.toileting.tsv
19909 (1440,)
./data-har//DAY_19909/act/loc.3d57.toileting.tsv
19909 (1440,)
./data-har//DAY_19909/act/loc.ed9c.toileting.tsv
19909 (1440,)
19910 ['0001', '3d57', 'ed9c']
./data-har//DAY_19910/act/loc.0001.toileting.tsv
19910 (1440,)
./data-har//DAY_19910/act/loc.3d57.toileting.tsv
19910 (1440,)
./data-har//DAY_19910/act/loc.ed9c.toileting.tsv
19910 (1440,)
19911 ['0001', '3d57', 'ed9c']
./data-har//DAY_19911/act/loc.0001.toileting.tsv
19911 (1440,)
./data-har//DAY_19911/act/loc.3d57.toileting.tsv
19911 (1440,)
./data-har//DAY_19911/act/loc.ed9c.toileting.tsv
19911 (1440,)
19922 ['16fe', 'f02e', 'ed9c']
./data-har//DAY_19922/act/loc.16fe.toileting.tsv
19922 (1440,)
./data-har//DAY_19922/act/loc.f02e.toileting.tsv
19922 (1440,)
./data-har//DAY_19922/act/loc.ed9c.toileting.tsv
19922 (1440,)
19923 ['16fe', 'f02e', 'ed9c']
./data-har//DAY_19923/act/loc.16fe.toileting.tsv
19923 (1440

In [5]:
act_sensors=np.zeros((len(sensors),len(days),sizeT(t0,tN)))
print(act_sensors.shape)
for sx,sensor in enumerate(sensors):
    print("\t",sensor)
    for dx,day in enumerate(days):
        print(processed_data_folder+"/DAY_"+str(day)+"/object_sensor/"+sensor+".tsv")
        series=read_csv(processed_data_folder+"/DAY_"+str(day)+"/object_sensor/"+sensor+".tsv", sep="\t",parse_dates=True, header=None, names=["time","status"])
        series=series["status"].values
        print(day,series.shape)
        act_sensors[sx][dx]=series

(11, 14, 1440)
	 fridge_13
./data-har//DAY_19909/object_sensor/fridge_13.tsv
19909 (1440,)
./data-har//DAY_19910/object_sensor/fridge_13.tsv
19910 (1440,)
./data-har//DAY_19911/object_sensor/fridge_13.tsv
19911 (1440,)
./data-har//DAY_19922/object_sensor/fridge_13.tsv
19922 (1440,)
./data-har//DAY_19923/object_sensor/fridge_13.tsv
19923 (1440,)
./data-har//DAY_19924/object_sensor/fridge_13.tsv
19924 (1440,)
./data-har//DAY_19928/object_sensor/fridge_13.tsv
19928 (1440,)
./data-har//DAY_19929/object_sensor/fridge_13.tsv
19929 (1440,)
./data-har//DAY_19930/object_sensor/fridge_13.tsv
19930 (1440,)
./data-har//DAY_19931/object_sensor/fridge_13.tsv
19931 (1440,)
./data-har//DAY_19932/object_sensor/fridge_13.tsv
19932 (1440,)
./data-har//DAY_19933/object_sensor/fridge_13.tsv
19933 (1440,)
./data-har//DAY_19934/object_sensor/fridge_13.tsv
19934 (1440,)
./data-har//DAY_19935/object_sensor/fridge_13.tsv
19935 (1440,)
	 tap_22
./data-har//DAY_19909/object_sensor/tap_22.tsv
19909 (1440,)
./data-

In [6]:
user_activity=np.zeros((len(activities),len(users),len(days),sizeT(t0,tN)))
for dx,day in enumerate(days):
    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,users)
    (t0,tN)=t0N_days[day]
    ts=list(range(t0,tN,time_step2))
    for ax,activity in enumerate(activities):
        for ux,user in enumerate(users):
            df_total=read_csv(processed_data_folder+"/DAY_"+str(day)+"/act/INTERVAL."+user+"."+activity+".tsv", sep="\t",parse_dates=True, header=None, names=["ix","d0","dN"])
            #print(df_total)
            #print(pd.to_datetime(df_total['d0']))
            df_total['d0'] = pd.to_datetime(df_total['d0'])
            df_total['dN'] = pd.to_datetime(df_total['dN'])
            #print(df_total)

            for tx,tt in enumerate(ts):
                ttt=pd.to_datetime(time2str(t0,tN,tt))
#                print(ttt)
                row = df_total[(df_total['d0'] <= ttt) & (df_total['dN'] >= ttt)]
#                print("df_in#tervalo",row)
                if(row.size>0):
                    user_activity[ax][ux][dx][tx]=1

for dx,day in enumerate(days):
    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,users)
    (t0,tN)=t0N_days[day]
    ts=list(range(t0,tN,time_step2))
            
    for ax,activity in enumerate(activities):
        for ux,user in enumerate(users):
            with open(processed_data_folder+"/DAY_"+str(day)+"/act/LABEL3."+user+"."+activity+".tsv", "w") as file:
                for tx,tt in enumerate(ts):
                    file.write(time2str(t0,tN,tt)+"\t"+str(user_activity[ax][ux][dx][tx])+"\n")




19909 ['0001', '3d57', 'ed9c']
19910 ['0001', '3d57', 'ed9c']
19911 ['0001', '3d57', 'ed9c']
19922 ['16fe', 'f02e', 'ed9c']
19923 ['16fe', 'f02e', 'ed9c']
19924 ['16fe', 'f02e', 'ed9c']
19928 ['16fe', '5b66', 'ed9c']
19929 ['16fe', '5b66', 'ed9c']
19930 ['16fe', '5b66', 'ed9c']
19931 ['16fe', '5b66', 'ed9c']
19932 ['16fe', '5b66', 'ed9c']
19933 ['16fe', '5b66', 'ed9c']
19934 ['16fe', '5b66', 'ed9c']
19935 ['16fe', '5b66', 'ed9c']
19909 ['0001', '3d57', 'ed9c']
19910 ['0001', '3d57', 'ed9c']
19911 ['0001', '3d57', 'ed9c']
19922 ['16fe', 'f02e', 'ed9c']
19923 ['16fe', 'f02e', 'ed9c']
19924 ['16fe', 'f02e', 'ed9c']
19928 ['16fe', '5b66', 'ed9c']
19929 ['16fe', '5b66', 'ed9c']
19930 ['16fe', '5b66', 'ed9c']
19931 ['16fe', '5b66', 'ed9c']
19932 ['16fe', '5b66', 'ed9c']
19933 ['16fe', '5b66', 'ed9c']
19934 ['16fe', '5b66', 'ed9c']
19935 ['16fe', '5b66', 'ed9c']


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, GRU, Dropout, Concatenate, Reshape
from tensorflow.keras.models import Model

# Assuming N1, N2, N3 are defined previously representing the length of activities and sensors
N1 = len(activities)
N2 = len(activities)
N3 = len(sensors)

TWA = 45
TWB = 0
T = TWA + TWB

def getModelHAR():
    # Define inputs
    input1 = Input(shape=(N1, T, 1))
    input3 = Input(shape=(N3, T, 1))

    # Reshape inputs to match the expected input shape of GRU layers
    reshaped1 = Reshape((N1, T))(input1)
    reshaped3 = Reshape((N3, T))(input3)

    # GRU block for the first sequence
    gru1 = GRU(128, return_sequences=True)(reshaped1)


    # GRU block for the third sequence
    gru3 = GRU(128, return_sequences=True)(reshaped3)

    # Concatenation of the three GRU blocks
    merged = Concatenate(axis=1)([gru1, gru3])
    merged = Dropout(0.5)(merged)

    # First GRU layer after concatenation
    gru_merged1 = GRU(256, return_sequences=True)(merged)
    gru_merged1 = Dropout(0.5)(gru_merged1)

    # Second GRU layer after concatenation
    gru_merged2 = GRU(512)(gru_merged1)  # return_sequences=False to prepare output for dense layers
    gru_merged2 = Dropout(0.5)(gru_merged2)

    # Dense layer for binary classification
    dense = Dense(128, activation='relu')(gru_merged2)
    dense = Dropout(0.5)(dense)
    dense = Dense(32, activation='relu')(dense)
    output = Dense(1, activation='sigmoid')(dense)

    # Define the model
    model = Model(inputs=[input1, input3], outputs=output)

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Instantiate the model
model = getModelHAR()
model.summary()


2024-12-07 12:39:51.544854: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 12:39:51.578536: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-07 12:39:52.454208: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-12-07 12:39:54.009254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-07 1

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8, 45, 1)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 11, 45, 1)]  0           []                               
                                                                                                  
 reshape (Reshape)              (None, 8, 45)        0           ['input_1[0][0]']                
                                                                                                  
 reshape_1 (Reshape)            (None, 11, 45)       0           ['input_2[0][0]']                
                                                                                              

In [8]:
import random



SizeTrainOk=4000
SizeTrainKo=6000


for dx,day in enumerate(days):
    print("\t",dx,day)
    other_dx= [other for other in range(0,len(days)) if other != dx]

    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,users)
    
    for ax,activity in enumerate(activities):
            print(ax,activity)
            
            ##TRAING DATA
            L1=[]
            L2=[]
            S=[]
            Y=[]
            for ux,user in enumerate(users):
                    print("\t\t",ux,user)
                    other_ux= [other for other in range(0,len(users)) if other != ux]
                    print(day,user,other_dx,other_ux)

                    #OK act
                    for ix in range(0,SizeTrainOk):
                        #print("SizeTrainOk",ix,"vs",SizeTrainOk)
                        filtered_elements = user_activity[ax, ux, other_dx, :]

                        # Encontramos los elementos que son 1
                        indices = np.argwhere(filtered_elements == 1)

                        if(len(indices)==0):
                            #print("\t\t\t","Non activity for user ")
                            break
                        
                        # Para obtener las posiciones relativas a las dimensiones originales
                        indexs = [(ax, ux, other_dx[idx[0]], idx[1]) for idx in indices]
                        index = random.choice(indexs)
                        #print("\t\t\t",index,user_activity[index])
                        
                        (ia,iu,idy,tt)=index
                        
                        if(tt<TWA):
                            tt=TWA
                        if(tt>=sizeT(t0,tN)-TWB):
                            tt=sizeT(t0,tN)-TWB
                        
                        
                        X1s=[]
                        X2s=[]
                        for ax2,activity2 in enumerate(activities):
                            X1s.append(loc_user[ax2][iu][idy][tt-TWA:tt+TWB])
                            X2s.append(np.amax(loc_user[ax2,other_ux,idy,tt-TWA:tt+TWB],axis=0))
                            
                            
                        Ss=[]
                        for sx, sensor in enumerate(sensors):
                            Ss.append(act_sensors[sx][idy][tt-TWA:tt+TWB])
                        L1.append(X1s)
                        L2.append(X2s)
                        S.append(Ss)
                        Y.append(user_activity[index])
                        
                    #KO act
                    ix=0
                    while ix < SizeTrainKo:
                        #print("SizeTrainKo",ix,"vs",SizeTrainKo)
                        filtered_elements_specific = user_activity[ax, ux, other_dx, :]

                        # Encontramos los elementos que son 1
                        indices = np.argwhere(filtered_elements == 0)

                        if(len(indices)==0):
                            print("\t\t\t","Non activity for user ")
                            break
                        
                        # Para obtener las posiciones relativas a las dimensiones originales
                        indexs = [(ax, ux, other_dx[idx[0]], idx[1]) for idx in indices]
                        index = random.choice(indexs)
                        #print("\t\t\t",index,user_activity[index])
                        
                        (ia,iu,idy,tt)=index
                        
                        if(tt<TWA):
                            tt=TWA
                        if(tt>=sizeT(t0,tN)-TWB):
                            tt=sizeT(t0,tN)-TWB
                        
                        
                        X1s=[]
                        X2s=[]
                        none=True
                        for ax2,activity2 in enumerate(activities):
                            #if(ax2==ax):
                                r=np.random.rand()
                                #print(" removing loc?:",loc_user[ax2][iu][idy][tt],r)
                                if(r<loc_user[ax2][iu][idy][tt]+0.05):
                                    #print("NOT  removing loc:",loc_user[ax2][iu][idy][tt])
                                    none=False
                                    
                                X1s.append(loc_user[ax2][iu][idy][tt-TWA:tt+TWB])
                                X2s.append(np.amax(loc_user[ax2,other_ux,idy,tt-TWA:tt+TWB],axis=0))
                            
                            
                        if not none:
                            L1.append(X1s)
                            L2.append(X2s)
                            Ss=[]
                            for sx, sensor in enumerate(sensors):
                                Ss.append(act_sensors[sx][idy][tt-TWA:tt+TWB])
                            S.append(Ss)
                            Y.append(user_activity[index])       
                            ix=ix+1

            L1=np.array(L1)        
            L2=np.array(L2)        
            S=np.array(S)
            Y=np.array(Y)
            print("TRAIN",activity,day,L1.shape,L2.shape,S.shape,Y.shape)
            
            
            model=getModelHAR()
            model.summary()
            
            ##TEST DATA
            TL1=[]
            TL2=[]
            TS=[]
            TY=[]
            
            
            (t0,tN)=t0N_days[day]
            ts=list(range(TWA,sizeT(t0,tN)-TWB))
            
            for ux2,user2 in enumerate(users):
                other_ux2= [other for other in range(0,len(users)) if other != ux2]

                for tx in ts:

                    Ss=[]
                    for sx, sensor in enumerate(sensors):
                        Ss.append(act_sensors[sx][dx][tx-TWA:tx+TWB])
                    
                    X1s=[]
                    X2s=[]
                    for ax2,activity2 in enumerate(activities):
                        X1s.append(loc_user[ax2][ux2][dx][tx-TWA:tx+TWB])
                        X2s.append(np.amax(loc_user[ax2,other_ux2,dx,tx-TWA:tx+TWB],axis=0))


                    TL1.append(X1s)
                    TL2.append(X2s)
                    TS.append(Ss)
                    TY.append(user_activity[ax, ux2, dx, tx])
                        
            TL1=np.array(TL1)        
            TL2=np.array(TL2)        
            TS=np.array(TS)
            TY=np.array(TY)
            print("EVAL",activity,day,TL1.shape,TL2.shape,TS.shape,TY.shape)
            
            history = model.fit([L1,S] , Y, shuffle=True, batch_size=32,epochs=10, validation_data = ([TL1,TS], TY))
            YT_prediction = model.predict([TL1,TS])
            print(YT_prediction.shape)
            
            ix=0
            for ux2,user2 in enumerate(users):
                with open(processed_data_folder+"/DAY_"+str(day)+"/act/PREDICTION-GRU-other-45+0."+user2+"."+activity+".tsv", "w") as file:
                    for tx in (ts):
                        tt=tx*time_step2+t0
                        #print("\t"+time2str(t0,tN,tt)+"\t"+str(TY[ix])+"\t"+str(YT_prediction[ix][0]))
                        file.write("\t"+time2str(t0,tN,tt)+"\t"+str(TY[ix])+"\t"+str(YT_prediction[ix][0])+"\n")
                        
                        ix=ix+1




	 0 19909
19909 ['0001', '3d57', 'ed9c']
0 toileting
		 0 0001
19909 0001 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] [1, 2]
		 1 3d57
19909 3d57 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] [0, 2]
		 2 ed9c
19909 ed9c [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] [0, 1]
TRAIN toileting 19909 (30000, 8, 45) (30000, 8, 45) (30000, 11, 45) (30000,)


2024-12-07 12:41:10.651565: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-07 12:41:10.652830: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-12-07 12:41:10.654024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 8, 45, 1)]   0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 11, 45, 1)]  0           []                               
                                                                                                  


2024-12-07 12:41:10.873474: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-07 12:41:10.874271: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-12-07 12:41:10.876809: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

 reshape_2 (Reshape)            (None, 8, 45)        0           ['input_3[0][0]']                
                                                                                                  
 reshape_3 (Reshape)            (None, 11, 45)       0           ['input_4[0][0]']                
                                                                                                  
 gru_4 (GRU)                    (None, 8, 128)       67200       ['reshape_2[0][0]']              
                                                                                                  
 gru_5 (GRU)                    (None, 11, 128)      67200       ['reshape_3[0][0]']              
                                                                                                  
 concatenate_1 (Concatenate)    (None, 19, 128)      0           ['gru_4[0][0]',                  
                                                                  'gru_5[0][0]']                  
          

...

938/938 [==============================] - ETA: 0s - loss: 0.2575 - accuracy: 0.8979

2024-12-07 19:47:41.445020: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-07 19:47:41.446977: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-12-07 19:47:41.447975: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

938/938 [==============================] - 20s 17ms/step - loss: 0.2575 - accuracy: 0.8979 - val_loss: 0.2193 - val_accuracy: 0.8951
Epoch 2/10
938/938 [==============================] - 15s 16ms/step - loss: 0.1779 - accuracy: 0.9323 - val_loss: 0.1778 - val_accuracy: 0.9140
Epoch 3/10
938/938 [==============================] - 15s 16ms/step - loss: 0.1538 - accuracy: 0.9444 - val_loss: 0.1914 - val_accuracy: 0.8703
Epoch 4/10
938/938 [==============================] - 16s 17ms/step - loss: 0.1406 - accuracy: 0.9487 - val_loss: 0.1529 - val_accuracy: 0.9367
Epoch 5/10
938/938 [==============================] - 15s 16ms/step - loss: 0.1278 - accuracy: 0.9537 - val_loss: 0.1383 - val_accuracy: 0.9381
Epoch 6/10
938/938 [==============================] - 15s 16ms/step - loss: 0.1158 - accuracy: 0.9579 - val_loss: 0.1387 - val_accuracy: 0.9407
Epoch 7/10
938/938 [==============================] - 15s 16ms/step - loss: 0.1044 - accuracy: 0.9629 - val_loss: 0.2395 - val_accuracy: 0.9047
Epo

2024-12-07 19:50:00.751400: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-07 19:50:00.752233: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-12-07 19:50:00.754731: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

131/131 [==============================] - 1s 6ms/step
(4185, 1)
